# Proyecto 1
Daniel Machic 22118 - Maria José - Javier Chen

## Obtención de Datos

In [ ]:
import pandas as pd
import os
import re
from IPython.display import display, FileLink
import unicodedata
def clean_text(text):
    if isinstance(text, str):
        text = re.sub(r'[\x00-\x1F\x7F]', '', text)
        text = text.replace('"', "'").replace('“', "'").replace('”', "'")
    return text

def limpiar_filas_finales(df):
    """Elimina el patrón específico del final de los archivos"""
    # Identificar filas completamente vacías
    filas_vacias = df.isnull().all(axis=1)

    # Buscar el inicio del patrón final (2 vacías + texto + 1 vacía + copyright)
    inicio_pie = None
    for i in range(len(df)-4, -1, -1):  # Busca desde el final
        # Verifica el patrón: vacía, vacía, texto, vacía, copyright
        if (filas_vacias[i] and
            filas_vacias[i+1] and
            'Establecimientos encontrados' in str(df.iloc[i+2].values) and
            filas_vacias[i+3] and
            'Ministerio de Educaci' in str(df.iloc[i+4].values)):
            inicio_pie = i
            break

    # Si encontramos el patrón, cortamos el DataFrame
    if inicio_pie is not None:
        return df.iloc[:inicio_pie]
    return df

# Configuración principal
ruta_carpeta = "/Users/danielmachic/Desktop/Data Science/Proyecto1/Departamentos/"

if not os.path.exists(ruta_carpeta):
    raise FileNotFoundError(f"Carpeta no encontrada: {ruta_carpeta}")

archivos = [f for f in os.listdir(ruta_carpeta) if f.lower().endswith(('.xls', '.xlsx'))]
if not archivos:
    raise ValueError("No se encontraron archivos Excel")

print(f"Procesando {len(archivos)} archivos...")

dataframes = []
for archivo in archivos:
    try:
        ruta_completa = os.path.join(ruta_carpeta, archivo)
        engine = 'openpyxl' if archivo.lower().endswith('.xlsx') else 'xlrd'
        df = pd.read_excel(ruta_completa, skiprows=27, engine=engine)

        # Limpieza de filas finales
        df = limpiar_filas_finales(df)

        # Limpieza de datos
        df = df.rename(columns={'DEPARTAMENTO': 'Departamento'})
        for col in df.select_dtypes(include=['object']).columns:
            df[col] = df[col].apply(clean_text)

        dataframes.append(df)
        print(f"✔ {archivo} procesado (filas conservadas: {len(df)})")

    except Exception as e:
        print(f"✖ Error en {archivo}: {str(e)}")
        continue

# Consolidación y guardado (igual que antes)
if dataframes:
    df = pd.concat(dataframes, ignore_index=True)

    print("\nResumen final:")
    print(f"- Total registros: {len(df)}")
    print(f"- Departamentos únicos: {df['Departamento'].nunique()}")
    print("\nMuestra de datos:")
    display(df.head(10))

    try:
        output_excel = os.path.join(ruta_carpeta, "../datos_consolidados.xlsx")
        df.to_excel(output_excel, index=False, engine='openpyxl')
        print(f"\n Archivo guardado en: {output_excel}")
        display(FileLink(output_excel))
    except Exception as e:
        output_csv = os.path.join(ruta_carpeta, "../datos_consolidados.csv")
        df.to_csv(output_csv, index=False, encoding='utf-8-sig')
        print(f"\n⚠  Se guardó como CSV: {output_csv}")
        display(FileLink(output_csv))
else:
    print(" No se procesaron archivos válidos")

## Limpieza de Datos

#### Eliminación de columnas vacías

In [ ]:
df = df.drop(columns=['Unnamed: 1', 'Unnamed: 18', 'Unnamed: 19'])
print(df.columns.tolist())  # Para ver los nombres exactos de las columnas

Se eliminaron 3 columnas ya que a la hora de hacer la unión de todos los archivos, se crearon 3 columnas sin información.

## Columna Distrito

In [ ]:
#Valores vacíos en la columna Distrito
filas_vacias = df[df['DISTRITO'].isna()]
num_filas = len(filas_vacias)
print("Número de filas:", num_filas)

No se eliminarán las filas con valores vacíos en la columna Distrito, ya que podrían afectar el análisis. Actualmente existen 525 valores vacíos, por lo que, en lugar de eliminarlos, se procederá a modificar la información, no solo en esa columna, sino también en otras.

In [ ]:
df['DISTRITO'] = df['DISTRITO'].fillna('NO ESPECIFICADO')

In [ ]:
conteo = (df['DISTRITO'] == '01-').sum()
print(f"El valor '01-' aparece {conteo} veces en la columna DISTRITO.")

# Reemplazar '01-' por 'NO ESPECIFICADO' en la columna DISTRITO
df['DISTRITO'] = df['DISTRITO'].replace('01-', 'NO ESPECIFICADO')


El valor "01-" aparecía 66 veces y representaba un dato incompleto; por ello, se decidió reemplazarlo por "NO ESPECIFICADO", al igual que los valores vacíos, con el fin de no afectar el análisis.

## Columnas Departamento y MUNICIPIO

In [ ]:
columnas = ['Departamento', 'MUNICIPIO']
df[columnas] = df[columnas].apply(lambda x: x.str.upper().str.replace(r'\s+', ' ', regex=True).str.strip())

Se asegura que las columnas tengan un mismo formato y se elimina cualquier espacio al inicio o al final

## Columna Establecimiento

In [ ]:
df = df.dropna(subset=['ESTABLECIMIENTO'])

Se eliminan las columnas donde "Establecimiento" no tiene información ya que al revisar el campo de "Status", estas se encuentran cerradas definitivamente. Además en el campo "Distrito" estas no cuentan con información al igual que en el campo de "Director", estos siendo campos importantes en la base de datos

In [ ]:
# Diccionario de reemplazos personalizados
REEMPLAZOS = {
    "ӎ": "ÓN",
    "Ѵ": "V",
    "Ι": "I",
    "Ӑ": "Á",
    "Ӓ": "Ä",
    "ɢ": "É",
    "ӂ": "OB",
    "Ƀ": "ÉC",
    "Ӈ": "ÓG",
    "ڂ": "ÚB",
    "я": "ÑO",
    "Ɓ": "ÉS",
    "Ɏ": "ÉN",
    "ړ": "ÚS",
    "܅": "UE",
    "Ӎ": "ÓM",
    "Ɒ": "ÉR",
    "Ӌ": "ÓL",
    "ځ": "ÚA",
    "چ": "ÚF",
    "Ɠ": "É",
    "F͓": "ÍS",
    "A͢": "AÍ",
    "N͢": "NÍ",
    "R͓": "RÍS",
    "ɼ/TD>": "É",
    "T͓": "TÍS",
    "Ɍ": "ÉL",
    "T͎": "ÍN",
    "A͓": "ÍS",
    "Ͼ/TD>": "Í",
}

# Función para limpiar caracteres raros según el diccionario
def limpiar_caracteres(valor):
    valor = unicodedata.normalize("NFC", str(valor))
    for raro, correcto in REEMPLAZOS.items():
        valor = valor.replace(raro, correcto)
    return valor

# Función para normalizar (quitar acentos y diacríticos)
def normalizar_ascii(texto):
    nfkd = unicodedata.normalize("NFKD", texto)
    return "".join(c for c in nfkd if not unicodedata.combining(c))

# Función para eliminar caracteres y espacios innecesarios
def limpiar_basura(texto):
    texto = re.sub(r"[,\-\(\)'\.]", " ", texto)  # Quitar , - ( ) ' .
    texto = re.sub(r"\s+", " ", texto)           # Quitar espacios dobles
    return texto.strip()                         # Quitar espacios extremos

# Patrón para detectar caracteres válidos (incluye letras, números y acentos)
patron_normal = re.compile(r"^[A-Za-zÁÉÍÓÚÜáéíóúüÑñ0-9 _]+$")

nombres_raros = set()
cambios = set()
nombres_normalizados = set()

# Procesar solo la columna ESTABLECIMIENTOS
for idx, valor in df["ESTABLECIMIENTO"].items():
    valor = str(valor).strip()
    if valor == "" or valor.lower() == "nan":
        continue

    #  Reemplazar caracteres raros según el diccionario
    valor_limpio = limpiar_caracteres(valor)

    #  Quitar comillas, puntos y otros símbolos innecesarios
    valor_limpio = limpiar_basura(valor_limpio)

    #  Normalizar a mayúsculas
    nombre_norm = valor_limpio.upper()

    # Detectar caracteres raros
    if not patron_normal.match(valor_limpio):
        nombres_raros.add(valor)

    # Guardar cambios detectados
    if valor.upper() != nombre_norm:
        cambios.add(f"{valor} → {nombre_norm}")

    # Reemplazar valor en el DataFrame
    df.at[idx, "ESTABLECIMIENTO"] = nombre_norm

    # Guardar normalizados
    nombres_normalizados.add(nombre_norm)

print(f"{len(nombres_raros)} nombres con caracteres raros detectados")
print(f"{len(cambios)} cambios realizados\n")

Se limpia y estandariza la columna ESTABLECIMIENTO, se reemplazan caracteres raros, se eliminan símbolos innecesarios, se eliminan espacios innecesarios, quita acentos y convierte todo a mayúsculas.

## COLUMNA DIRECCIÓN

In [ ]:
REEMPLAZOS = {
    "ӎ": "ÓN",
    "Ѵ": "V",
    "Ι": "I",
    "Ӑ": "Á",
    "Ӓ": "Ä",
    "ɢ": "É",
    "ӂ": "OB",
    "Ƀ": "ÉC",
    "Ӈ": "ÓG",
    "ڂ": "ÚB",
    "я": "ÑO",
    "Ɓ": "ÉS",
    "Ɏ": "ÉN",
    "ړ": "ÚS",
    "܅": "UE",
    "Ӎ": "ÓM",
    "Ɒ": "ÉR",
    "Ӌ": "ÓL",
    "ځ": "ÚA",
    "چ": "ÚF",
    "Ɠ": "É",
    "F͓": "ÍS",
    "A͢": "AÍ",
    "N͢": "NÍ",
    "R͓": "RÍS",
    "ɼ/TD>": "É",
    "T͓": "TÍS",
    "Ɍ": "ÉL",
    "T͎": "ÍN",
    "A͓": "ÍS",
    "Ͼ/TD>": "Í",}

# Función para limpiar caracteres raros según el diccionario
def limpiar_caracteres(valor):
    valor = str(valor)
    valor = unicodedata.normalize("NFC", valor)
    for raro, correcto in REEMPLAZOS.items():
        valor = valor.replace(raro, correcto)
    return valor

# Función para eliminar caracteres y espacios innecesarios
def limpiar_basura(texto):
    texto = re.sub(r"[,\(\)'\.]", " ", texto)   # quitar , ( ) ' .
    texto = re.sub(r"\s+", " ", texto)          # quitar espacios dobles
    return texto.strip()

# Patrón para detectar caracteres válidos
patron_normal = re.compile(r"^[A-Za-zÁÉÍÓÚÜáéíóúüÑñ0-9 _\-]+$")

# Conjuntos para detectar cambios y caracteres raros
nombres_raros = set()
cambios = set()

# Bucle para limpiar la columna DIRECCION
for idx, valor in df["DIRECCION"].items():
    # Convertir a string y quitar espacios extremos; tratar NaN
    original = str(valor).strip() if not pd.isna(valor) else ""

    # Detectar valores vacíos, NaN, ceros o secuencias de guiones/espacios
    if original == "" or re.fullmatch(r"[-\s]{2,}", original):
        df.at[idx, "DIRECCION"] = "SIN DIRECCION"
        continue

    # Limpiar caracteres según el diccionario
    valor_limpio = limpiar_caracteres(original)

    # Limpiar símbolos y espacios innecesarios
    valor_limpio = limpiar_basura(valor_limpio)

    # Normalizar a mayúsculas
    nombre_norm = valor_limpio.upper()

    # Detectar caracteres no permitidos
    if not patron_normal.match(valor_limpio):
        nombres_raros.add(original)

    # Guardar cambios si el valor cambió
    if original.upper() != nombre_norm:
        cambios.add(f"{original} → {nombre_norm}")

    # Reemplazar en el DataFrame
    df.at[idx, "DIRECCION"] = nombre_norm

print(f"{len(nombres_raros)} nombres con caracteres raros detectados")
print(f"{len(cambios)} cambios realizados")


Se limpia y estandariza la columna DIRECCIÓN, se reemplazan caracteres raros, se eliminan símbolos innecesarios, se eliminan espacios innecesarios, quita acentos y convierte todo a mayúsculas. Sin embargo se conservan los guiónes ya que son esenciales en las direcciones.

## COLUMNA TELEFONO

In [ ]:
def split_phones(phone):
    phone = str(phone)
    # Extraer solo secuencias de exactamente 8 dígitos
    numbers = re.findall(r'\d{8}', phone)
    # Formatear cada número como XXXX-XXXX
    numbers = [f"{n[:4]}-{n[4:]}" for n in numbers]
    # Completar con "Sin información" si hay menos de 3
    while len(numbers) < 3:
        numbers.append("Sin información")
    return numbers[:3]

# Crear columnas nuevas desde TELEFONO
df[['TELEFONO_1', 'TELEFONO_2', 'TELEFONO_3']] = df['TELEFONO'].apply(lambda x: pd.Series(split_phones(x)))

# Eliminar duplicados de las columnas nuevas si ya estaban en el DataFrame
for col in ['TELEFONO_1', 'TELEFONO_2', 'TELEFONO_3']:
    if col in df.columns[df.columns.duplicated()].tolist():
        df = df.loc[:, ~df.columns.duplicated()]

# Quitar la columna original TELEFONO
df = df.drop(columns=['TELEFONO'])

# Reordenar: insertar después de DIRECCION
columnas = list(df.columns)
pos = columnas.index('DIRECCION') + 1
for col in reversed(['TELEFONO_1', 'TELEFONO_2', 'TELEFONO_3']):
    columnas.insert(pos, columnas.pop(columnas.index(col)))

df = df[columnas]


En la columna TELEFONO, se extraen únicamente los dígitos que tengan exactamente 8 caracteres, eliminando guiones, comas, barras, texto adicional o números incompletos. Cada número válido se formatea con el patrón XXXX-XXXX y se asigna de forma ordenada a las columnas TELEFONO_1, TELEFONO_2 y TELEFONO_3, ya que algunas celdas contienen más de un número y se decidió crear columnas adicionales para evitar la pérdida de información. Si no se encuentran suficientes números, las posiciones faltantes se completan con el texto "Sin información", garantizando uniformidad en el formato y la estructura de los datos.

## COLUMNAS SUPERVISOR Y DIRECTOR

In [ ]:
# Diccionario de reemplazos personalizados
REEMPLAZOS = {
    "ӎ": "ÓN",
    "Ѵ": "V",
    "Ι": "I",
    "Ӑ": "Á",
    "Ӓ": "Ä",
    "ɢ": "É",
    "ӂ": "OB",
    "Ƀ": "ÉC",
    "Ӈ": "ÓG",
    "ڂ": "ÚB",
    "я": "ÑO",
    "Ɓ": "ÉS",
    "Ɏ": "ÉN",
    "ړ": "ÚS",
    "܅": "UE",
    "Ӎ": "ÓM",
    "Ɒ": "ÉR",
    "Ӌ": "ÓL",
    "ځ": "ÚA",
    "چ": "ÚF",
    "Ɠ": "É",
    "F͓": "ÍS",
    "A͢": "AÍ",
    "N͢": "NÍ",
    "R͓": "RÍS",
    "ɼ/TD>": "É",
    "T͓": "TÍS",
    "Ɍ": "ÉL",
    "T͎": "ÍN",
    "A͓": "ÍS",
    "Ͼ/TD>": "Í",
}

def limpiar_caracteres(valor):
    valor = str(valor)
    valor = unicodedata.normalize("NFKC", valor)
    for raro, correcto in REEMPLAZOS.items():
        valor = valor.replace(raro, correcto)
    return valor

def limpiar_basura(texto):
    texto = re.sub(r"[,\-(\)'\.]", " ", texto)
    texto = re.sub(r"\s+", " ", texto)
    return texto.strip()

patron_normal = re.compile(r"^[A-Za-zÁÉÍÓÚÜáéíóúüÑñ0-9 _]+$")

columnas_a_limpieza = ["SUPERVISOR", "DIRECTOR"]

for col in columnas_a_limpieza:
    nombres_raros = set()
    cambios = set()

    for idx, valor in df[col].items():
        # Detectar NaN y convertir a cadena vacía
        if pd.isna(valor):
            valor_str = ""
        else:
            valor_str = str(valor).strip()

        # Asignar valores por defecto si está vacío o inválido
        if col == "SUPERVISOR" and valor_str == "":
            df.at[idx, col] = "SIN SUPERVISOR"
            continue
        elif col == "DIRECTOR" and (valor_str == "" or valor_str == "0"):
            df.at[idx, col] = "SIN DIRECTOR"
            continue

        # Limpiar caracteres raros y símbolos
        valor_limpio = limpiar_caracteres(valor_str)
        valor_limpio = limpiar_basura(valor_limpio)
        nombre_norm = valor_limpio.upper()

        # Guardar cambios y caracteres raros
        if not patron_normal.match(valor_limpio):
            nombres_raros.add(valor_str)
        if valor_str.upper() != nombre_norm:
            cambios.add(f"{valor_str} → {nombre_norm}")

        # Reemplazar valor en el DataFrame
        df.at[idx, col] = nombre_norm

    print(f"Columna '{col}': {len(nombres_raros)} nombres con caracteres raros detectados")
    print(f"Columna '{col}': {len(cambios)} cambios realizados\n")


Se normalizan  las columnas "SUPERVISOR" y "DIRECTOR" además los valores vacíos se reemplazan por "SIN SUPERVISOR" y "SIN DIRECTOR"  y se  normalizan las mayúsculas.

## DEMÁS COLUMNAS

In [ ]:
columnas =  ["NIVEL", "SECTOR", "AREA", "STATUS", "MODALIDAD", "JORNADA", "PLAN", "DEPARTAMENTAL"]
df[columnas] = df[columnas].apply(lambda x: x.str.upper().str.replace(r'\s+', ' ', regex=True).str.strip())

Se normalizaron las columnas poniendolas en mayúsculas y quitando caracteres especiales

## COLUMNA PLAN

In [ ]:
# Unificar valores de PLAN que empiecen con "SEMIPRESENCIAL"
df["PLAN"] = df["PLAN"].apply(lambda x: "SEMIPRESENCIAL" if str(x).strip().upper().startswith("SEMIPRESENCIAL") else x)

Se unificaron todos los datos que contenían SEMIPRESENCIAL para un mejor manejo de los datos

## COLUMNA DEPARTAMENTAL

In [ ]:
# Diccionario de reemplazos para la columna DEPARTAMENTAL
REEMPLAZOS_DEPT = {
    "Ɏ": "ÉN",
    "ɼ/TD>": "É",
    "|/TD>": "Á",
    "ɑ": "ÉQ",
    "Ɠ": "É ",
    "Ɑ": "ÉQ",
}

def limpiar_departamental(valor):
    valor = str(valor).strip()
    for raro, correcto in REEMPLAZOS_DEPT.items():
        valor = valor.replace(raro, correcto)
    return valor

# Aplicar limpieza solo a la columna DEPARTAMENTAL
if "DEPARTAMENTAL" in df.columns:
    df["DEPARTAMENTAL"] = df["DEPARTAMENTAL"].apply(limpiar_departamental)


Se normalizó la columna DEPARTAMENTAL y se cambiaron los caracteres especiales

# Exportación de documento a Excel

In [ ]:
df.to_excel("datos_normalizadas.xlsx", index=False)

print("Archivo guardado como datos_normalizados.xlsx")